# Data analysis with Python, Apache Spark,  and PixieDust
***

In this notebook you will:

* analyze customer demographics, such as, age, gender, income, and location
* combine that data with sales data to examine trends for product categories, transaction types, and product popularity
* load data from GitHub as well as from a public open data set
* cleanse, shape, and enrich the data, and then visualize the data with the PixieDust library

Don't worry! PixieDust charts don't require coding. 

By the end of the notebook, you will understand how to combine data to gain insights about which customers you might target to increase sales.

<a id="toc"></a>
## Table of contents

#### [Setup](#Setup)
[Load data into the notebook](#Load-data-into-the-notebook)
#### [Explore customer demographics](#part1)
[Prepare the customer data set](#Prepare-the-customer-data-set)<br>
[Visualize customer demographics and locations](#Visualize-customer-demographics-and-locations)<br>

#### [Summary and next steps](#summary)

## Setup
You need to import libraries and load the customer data into this notebook.

Import the necessary libraries:

In [ ]:
import pixiedust
import pyspark.sql.functions as func
import pyspark.sql.types as types
import re
import json
import os
import requests  

**If you get any errors or if a package is out of date:**

* uncomment the lines in the next cell (remove the `#`)
* restart the kernel (from the Kernel menu at the top of the notebook)
* reload the browser page
* run the cell above, and continue with the notebook

In [ ]:
#!pip install pixiedust --upgrade

### Load data into the notebook

The data file contains both the customer demographic data that you'll analyzed in Part 1, and the sales transaction data for Part 2.

With `pixiedust.sampleData()` you can load csv data from any url. The below loads the data in a Spark DataFrame. 

> In case you wondered, this works with Pandas as well, just add `forcePandas = True` to load data in a Pandas DataFrame. *But do not add this to the below cell as in this notebook you will use Spark.*

In [ ]:
raw_df = pixiedust.sampleData('https://raw.githubusercontent.com/IBMCodeLondon/localcart-workshop/master/data/customers_orders1_opt.csv')

In [ ]:
raw_df

[Back to Table of Contents](#toc)
<a id="part1"></a>
# Explore customer demographics 
In this part of the notebook, you will prepare the customer data and then start learning about your customers by creating multiple charts and maps. 

## Prepare the customer data set
Create a new Spark DataFrame with only the data you need and then cleanse and enrich the data.

Extract the columns that you are interested in, remove duplicate customers, and add a column for aggregations:

In [ ]:
# Extract the customer information from the data set
customer_df = raw_df.select("CUST_ID", 
                            "CUSTNAME", 
                            "ADDRESS1", 
                            "ADDRESS2", 
                            "CITY", 
                            "POSTAL_CODE", 
                            "POSTAL_CODE_PLUS4", 
                            "STATE", 
                            "COUNTRY_CODE", 
                            "EMAIL_ADDRESS", 
                            "PHONE_NUMBER",
                            "AGE",
                            "GenderCode",
                            "GENERATION",
                            "NATIONALITY", 
                            "NATIONAL_ID", 
                            "DRIVER_LICENSE").dropDuplicates()

customer_df.printSchema()

Notice that the data type of the AGE column is currently a string. Convert the AGE column to a numeric data type so you can run calculations on customer age.

In [ ]:
# ---------------------------------------
# Cleanse age (enforce numeric data type) 
# ---------------------------------------

def getNumericVal(col):
    """
    input: pyspark.sql.types.Column
    output: the numeric value represented by col or None
    """
    try:
      return int(col)
    except ValueError:
      # age-33
      match = re.match('^age\-(\d+)$', col)
      if match:
        try:
          return int(match.group(1))
        except ValueError:    
          return None
      return None  

toNumericValUDF = func.udf(lambda c: getNumericVal(c), types.IntegerType())
customer_df = customer_df.withColumn("AGE", toNumericValUDF(customer_df["AGE"]))
customer_df

In [ ]:
customer_df.show(5)

The GenderCode column contains salutations instead of gender values. Derive the gender information for each customer based on the salutation and rename the GenderCode column to GENDER.

In [ ]:
# ------------------------------
# Derive gender from salutation
# ------------------------------
def deriveGender(col):
    """ input: pyspark.sql.types.Column
        output: "male", "female" or "unknown"
    """    
    if col in ['Mr.', 'Master.']:
        return 'male'
    elif col in ['Mrs.', 'Miss.']:
        return 'female'
    else:
        return 'unknown';
    
deriveGenderUDF = func.udf(lambda c: deriveGender(c), types.StringType())
customer_df = customer_df.withColumn("GENDER", deriveGenderUDF(customer_df["GenderCode"]))
customer_df.cache()

## Explore the customer data set

Instead of exploring the data with `.printSchema()` and `.show()` you can quickly explore data sets using PixieDust'. Invoke the `display()` command and click the table icon to review the schema and preview the data. Customize the options to display only a subset of the fields or rows or apply a filter (by clicking the funnel icon).

In [ ]:
display(customer_df)

[Back to Table of Contents](#toc)
## Visualize customer demographics and locations

Now you are ready to explore the customer base. Using simple charts, you can quickly see these characteristics:
 * Customer demographics (gender and age)
 * Customer locations (city, state, and country)

You will create charts with the PixieDust library:

 - [View customers by gender in a pie chart](#View-customers-by-gender-in-a-pie-chart)
 - [View customers by generation in a bar chart](#View-customers-by-generation-in-a-bar-chart)
 - [View customers by age in a histogram chart](#View-customers-by-age-in-a-histogram-chart)
 - [View specific information with a filter function](#View-specific-information-with-a-filter-function)
 - [View customer density by location with a map](#View-customer-density-by-location-with-a-map)

### View customers by gender in a pie chart

Run the `display()` command and then configure the graph to show the percentages of male and female customers:

1. Run the next cell. The PixieDust interactive widget appears.  
1. Click the chart button and choose **Pie Chart**. The chart options tool appears.
1. In the chart options, drag `GENDER` into the **Keys** box. 
1. In the **Aggregation** field, choose **COUNT**. 
1. Increase the **# of Rows to Display** to a very large number to display all data.
1. Click **OK**. The pie chart appears.

If you want to make further changes, click **Options** to return to the chart options tool.

In [ ]:
display(customer_df)

[Back to Table of Contents](#toc)
### View customers by generation in a bar chart
Look at how many customers you have per "generation."

Run the next cell and configure the graph: 
1. Choose **Bar Chart** as the chart type and configure the chart options as instructed below.
2. Put `GENERATION` into the **Keys** box.
3. Set **aggregation** to `COUNT`.
1. Increase the **# of Rows to Display** to a very large number to display all data.
4. Click **OK**
4. Change the **Renderer** at the top right of the chart to explore different visualisations.  
4. You can use clustering to group customers, for example by geographic location. To group generations by country, select `COUNTRY_CODE` from the **Cluster by** list from the menu on the left of the chart. 

In [ ]:
display(customer_df)

[Back to Table of Contents](#toc)
### View customers by age in a histogram chart
A generation is a broad age range. You can look at a smaller age range with a histogram chart. A histogram is like a bar chart except each bar represents a range of numbers, called a bin. You can customize the size of the age range by adjusting the bin size. The more bins you specify, the smaller the age range.

Run the next cell and configure the graph:
1. Choose **Histogram** as the chart type. 
2. Put `AGE` into the **Values** box.
1. Increase the **# of Rows to Display** to a very large number to display all data.
1. Click **OK**.
3. Use the **Bin count** slider to specify the number of the bins. Try starting with 40.

In [ ]:
display(customer_df)

[Back to Table of Contents](#toc)

PixieDust supports basic filtering to make it easy to analyse data subsets. For example, to view the age distribution for a specific gender configure the chart as follows:

  1. Choose `Histogram` as the chart type.
  2. Put `AGE` into the **Values** box and click OK.
  3. Click the filter button (looking like a funnel), and choose **GENDER** as field and `female` as value, and click `Apply`.
  
The filter is only applied to the working data set and does not modify the input `customer_df`.


In [ ]:
display(customer_df)

You can also filter by location. For example, the following command creates a new DataFrame that filters for customers from the USA:

In [ ]:
condition = "COUNTRY_CODE = 'US'"
us_customer_df = customer_df.filter(condition)

You can pivot your analysis perspective based on aspects that are of interest to you by choosing different keys and clusters.

Create a bar chart and cluster the data.

Run the next cell and configure the graph:
1. Choose **Bar chart** as the chart type.
2. Put `COUNTRY_CODE` into the **Keys** box.
4. Set Aggregation to **COUNT**.
5. Click **OK**. The chart displays the number of US customers.
6. From the **Cluster By** list, choose **GENDER**. The chart shows the number of customers by gender.

In [ ]:
display(us_customer_df)

Now try to cluster the customers by state.

A bar chart isn't the best way to show geographic location!

[Back to Table of Contents](#toc)
### View customer density by location with a map
Maps are a much better way to view location data than other chart types. 

Visualize customer density by US state with a map.

Run the next cell and configure the graph:
1. Choose **Map** as the chart type.
2. Put `STATE` into the **Keys** box.
4. Set Aggregation to **COUNT**.
5. Click **OK**. The map displays the number of US customers.
6. From the **Renderer** list, choose **brunel**.

    > PixieDust supports three map renderers: brunel, [mapbox](https://www.mapbox.com/) and Google. Note that the Mapbox renderer and the Google renderer require an API key or access token and supported features vary by renderer.

7. You can explore more about customers in each state by changing the aggregation method, for example look at customer age ranges (avg, minimum, and maximum) by state. Simply Change the aggregation function to `AVG`, `MIN`, or `MAX` and choose `AGE` as value. 


In [ ]:
display(us_customer_df)

[Back to Table of Contents](#toc)
### View data using matplotlib

PixieDust is a great tool for quick visualisations, but using the Python visualisation packages directly gives you more control over the charts you create. Below is a simple example of what you can do with matplotlib.
    
    

In [ ]:
# without this the plots would be opened  in a new window (not browser)
# with this instruction plots will be included in the notebook
%matplotlib inline

import matplotlib.pyplot as plt

Internally PixieDust converts the data to a Pandas DataFrame as this is what most visualisation packages use. So let's do the same here:

In [ ]:
df = customer_df.toPandas()
df.head()

The default plot is a line chart:

In [ ]:
df['AGE'].plot();

To create a plot that makes more sense for this data have a look at the [documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/visualization.html) for all options. A histogram might work better. Go ahead and change the number of bins until you think the number of bins looks right:

In [ ]:
df['AGE'].plot.hist(bins=8);

Change the size of the plot with `figsize`:

In [ ]:
df['AGE'].plot.hist(bins=15,figsize=(10,5));

The below plot shows only the womens age:

In [ ]:
df['AGE'][df['GENDER']=='female'].plot.hist(bins=15,figsize=(10,5));

To add the men, simply repeat the plot command with a different selection of the data:

In [ ]:
df['AGE'][df['GENDER']=='female'].plot.hist(bins=15,figsize=(10,5));
df['AGE'][df['GENDER']=='male'].plot.hist(bins=15,figsize=(10,5));

The above plot is difficult to read as the histograms overlap. You can fix this by changing the colours and making them transparant. To add a legend each histogram needs to be assigned to an object `ax` that is used to create a legend:

In [ ]:
ax = df['AGE'][df['GENDER']=='female'].plot.hist(
    bins=15,figsize=(10,5),alpha=0.5,color='#1A4D3B');
ax = df['AGE'][df['GENDER']=='male'].plot.hist(
    bins=15,figsize=(10,5),alpha=0.5,color='#4D1A39');
ax.legend(['female','male']);

It is easy to change pretty much everything as in the below code. This was the ugliest I could come up with. Can you make it worse?

In [ ]:
df['AGE'].plot.hist(
    bins=15, 
    title="Age",
    legend=False,
    fontsize=14,
    grid=False,
    linestyle='--',
    edgecolor='black',
    color='darkred',
    linewidth=3);

You can use `groupby()` in combination with a bar plot to visualize average age by country:

In [ ]:
country = df['AGE'].groupby(df['COUNTRY_CODE']).mean()
ax=country.plot.bar();
ax.set_ylabel('Age');

[Back to Table of Contents](#toc)


Copyright © 2017, 2018 IBM. This notebook and its source code are released under the terms of the MIT License.